In [ ]:
import os
import json
import pandas as pd
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
import time
import matplotlib.pyplot as plt
from googleapiclient.discovery import build

scopes = ["https://www.googleapis.com/auth/youtube.readonly"]

In [ ]:
from googleapiclient.discovery import build

with open('secret_clientid.txt', 'r') as file:
        DEVELOPER_KEY = file.read()
# Configurez votre clé API et initialisez l'API YouTube
api_key = DEVELOPER_KEY
youtube = build("youtube", "v3", developerKey=api_key)

# Paramètres initiaux pour la requête
video_id = "x7JnA8ssuqY"
comments_data = []
next_page_token = None

while True:
    response = youtube.commentThreads().list(
        part="snippet",
        videoId=video_id,
        maxResults=100,
        order="time",
        textFormat="plainText",
        pageToken=next_page_token
    ).execute()

    # Ajouter les commentaires récupérés à la liste
    for item in response.get("items", []):
        comment_info = item["snippet"]["topLevelComment"]["snippet"]
        comments_data.append({
            "author": comment_info.get("authorDisplayName"),
            "publishedAt": comment_info.get("publishedAt"),
            "comment": comment_info.get("textDisplay")
        })

    # Vérifier s'il y a une page suivante
    next_page_token = response.get("nextPageToken")
    if not next_page_token:
        break
    time.sleep(1)

# Création du DataFrame à partir des données collectées
df = pd.DataFrame(comments_data)
print(df.head())
print(f"Total de commentaires récupérés : {len(comments_data)}")
